# 8.1 어텐션의 구조

- 어텐션 메커니즘, seq2seq는 필요한 정보에만 주목할 수 있게 되었다.

## 8.1.1 seq2seq의 문제점

- '고정 길이'라는데에 큰 문제가 잠재해 있다.
- 입력 문장의 길에 관계없이 항상 같은 길이의 벡터로 변환한다는 뜻

<img src="../../../data/deep_learning_2_images/fig 8-1.png" height="200">

- 많은 옷가지를 옷장에 욱여넣듯이 억지로 고정 길이의 벡터로 밀어 넣는 꼴

## 8.1.2 Encoder 개선

<img src="../../../data/deep_learning_2_images/fig 8-2.png" height="400">

- 기존 : 마지막 은닉 상태만을 Decoder에 전달
- Encoder 출력의 길이는 입력 문장의 길에 따라 바꿔주는게 좋다.
- LSTM 계층의 은닉 상태 벡터를 모두 이용
- 입력된 단어와 같은 수의 벡터를 얻을 수 있음
- '하나의 고정 길이 벡터'라는 제약으로부터 해방됨
- LSTM 계층의 은닉 상태의 '내용'
  - 각 시각의 은닉 상태에는 직전에 입력된 단어에 대한 정보가 많이 포함되어 있다.
  - $h_t$에는 $x_t$에 대한 정보가 많이 포함됨
  - Encoder가 출력하는 hs행렬은 각 단어에 해당하는 벡터들의 집합 

<img src="../../../data/deep_learning_2_images/fig 8-3.png" height="300">

- 주변 정보를 균형있게 담아야 할 때, 시계열 데이터를 양방향으로 처리해야할 떄
- 양방향 RNN or 양방향 LSTM이 효과적


## 8.1.3 Decoder 개선 1

<img src="../../../data/deep_learning_2_images/fig 8-4.png" height="200">
<img src="../../../data/deep_learning_2_images/fig 8-5.png" height="200">

- hs 전부를 이용할 수 있도록 Decoder를 개선
- 사람: 어떤 단어(혹은 단어의 집합)에 주목하여 그 단어의 변환을 수시로 하게 됨
- 입력과 출력의 여러 단어 중 어떤 단어끼리 서로 관련되어 있는가 라는 대응 관계를 학습시킬 수 없을까?
- alignment
  - 고양이 = cat
  - 단어의 대응관계를 나타내는 작업
  - 수작업 -> 기계에 의한 자동화

- 목표는 '도착어 단어'와 대응관계에 있는 '출발 단어'의 정보를 골라내는 것.
- 이를 이용해 번역을 수행하는 것
- 필요한 정보에만 주목하여 그 정보로부터 시계열 변환을 수행하는 것이 목표 
- 어텐션

<img src="../../../data/deep_learning_2_images/fig 8-6.png" height="400">

- 어떤 계산을 수행하는 계층을 추가
- 받는 입력은 2가지
  - Encoder로부터 받는 hs
  - 시각별 LSTM 계층의 은닉 상태
- 이 중 필요한 정보만 골라 위쪽의 Affine 계층으로 출력
- Encoder의 마지막 은닉 상태 벡터는 Decoder의 첫 번째 LSTM 계층에 전달

- 단어들의 얼라인먼트 추출
  - 각 시각에서 Decoder에 입력된 단어와 대응 관계인 단어의 벡터를 hs에서 골라내겠다.
  - '선택'작업을 '어떤 계산'으로 해내겠다.

- 문제 발생
  - 선택하는 작업은 미분할 수 없다는 점 -> 오차역전파 사용 불가능

<img src="../../../data/deep_learning_2_images/fig 8-7.png" height="200">

- 하나를 선택하는게 아니라 모든 것을 선택한다. 그리고 각 단어의 중요도를 나타내는 가중치를 별도로 계산

<img src="../../../data/deep_learning_2_images/fig 8-8.png" height="200">

- 맥락 벡터 c에는 현 시각의 변환(번역)을 수행하는데 필요한 정보가 담겨있다.  
(정확히는 그렇게 되도록 데이터로부터 학습한다.)

가중합 예시

<img src="../../../data/deep_learning_2_images/fig 8-9.png" height="150">
<img src="../../../data/deep_learning_2_images/fig 8-10.png" height="150">

In [7]:
import numpy as np

T, H = 5, 4
hs = np.random.randn(T, H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

# ar = a.reshape(5, 1).repeat(4, axis=1)
ar = a.reshape(5, 1)
print(ar.shape)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=0)
print(c.shape)

(5, 1)
(5, 4)
(4,)


가중합 계산 그래프

<img src="../../../data/deep_learning_2_images/fig 8-11.png" height="300">

In [8]:
class WeightSum:
  def __init__(self):
    self.params, self.grads = [], []
    self.cache = None
    
  def forward(self, hs, a):
    N, T, H = hs.shape
    
    ar = a.reshape(N, T, 1).repeat(H, axis=2)
    t = hs * ar
    c = np.sum(t, axis=1)
    
    self.cache = (hs, ar)
    
    return c
  
  def backward(self, dc):
    hs, ar = self.cache
    N, T, H = hs.shape
    
    dt = dc.reshape(N, 1, H).repeat(T, axis=1)
    dar = dt * hs
    dhs = dt * ar
    da = np.sum(dar, axis=2)
    
    return dhs, da
    

## 8.1.4 Decoder 개선 2

- 가중치 a가 있다면 , 가중합을 이용해 '맥락 벡터' 얻을 수 있음

<img src="../../../data/deep_learning_2_images/fig 8-12.png" height="400">

- h가 hs의 각 단어 벡터와 얼마나 '비슷한가'를 수치로 나타내는 것
- 내적 이용 , 두 벡터가 얼마나 같은 방향을 향하고 있는가

<img src="../../../data/deep_learning_2_images/fig 8-13.png" height="400">
<img src="../../../data/deep_learning_2_images/fig 8-14.png" height="400">


In [10]:
import sys
sys.path.append('../../modules/Part2/')
from common.layers import Softmax
import numpy as np

N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
# hr = h.reshape(N, 1, H).repeat(T, axis=1)
hr = h.reshape(N, 1, H)

t = hs * hr

s = np.sum(t, axis=2)

가중치 계산 그래프

<img src="../../../data/deep_learning_2_images/fig 8-15.png" height="300">


In [11]:
import sys
sys.path.append('../../modules/Part2/')
from common.layers import Softmax
import numpy as np

class AttentionWeight:
  def __init__(self):
    self.params, self.grads = [], []
    self.softmax = Softmax()
    self.cache = None
    
  def forward(self, hs, h):
    N, T, H = hs.shape
    
    hr = h.reshape(N, 1, H)
    t = hs * hr
    s = np.sum(t, axis=2)
    
    a = self.softmax.forward(s)
    
    self.cache = (hs, hr)
    
    return a
  
  def backward(self, da):
    hs, hr = self.cache
    N, T, H = hs.shape
    
    ds = self.softmax.backward(da)
    dt = ds.reshape(N, T, 1).repeat(H, axis=2)
    dhs = dt * hr
    dhr = dt * hs
    dh = np.sum(dhr, axis=1)
    
    return dhs, dh

## 8.1.5 Decoder 개선 3

- weight sum + attention weight

<img src="../../../data/deep_learning_2_images/fig 8-16.png" height="300">

- Attention Weight , Encoder가 출력하는 각 단어의 벡터 hs에 주목하여 단어의 가중치 a를 구함
- Weight Sum 계층이 a와 hs의 가중합을 구하고, 그 결과를 맥락 벡터 c로 출력

<img src="../../../data/deep_learning_2_images/fig 8-17.png" height="300">

- Encoder가 건네주는 정보 hs에서 중요한 원소에 주목하여 그것을 바탕으로 맥락 벡터를 구해 위쪽 계층으로 전파

In [12]:
class Attention:
  def __init__(self):
    self.params, self.grads = [], []
    self.attention_weight_layer = AttentionWeight()
    self.weight_sum_layer = WeightSum()
    self.attention_weight = None
    
  def forward(self, hs, h): # 맥락 벡터 구함
    a = self.attention_weight_layer.forward(hs, h)
    out = self.weight_sum_layer.forward(hs, a)
    self.attention_weight = a
    return out
  
  def backward(self, dout):
    dhs0, da = self.weight_sum_layer.backward(dout)
    dhs1, dh = self.attention_weight_layer.backward(da)
    dhs = dhs0 + dhs1
    return dhs, dh

전체 구조

<img src="../../../data/deep_learning_2_images/fig 8-18.png" height="500">
<img src="../../../data/deep_learning_2_images/fig 8-19.png" height="300">

- Decoder에 Attention 계층이 구한 맥락 벡터 정보를 '추가'한 것으로 생각할 수 있다.
- Affine 계층에는 기존과 마찬가지로 LSTM 계층의 은닉 상태 벡터를 주고,   
여기에 더해 Attention 계층의 맥락 벡터까지 입력


Time Attention으로 구축

<img src="../../../data/deep_learning_2_images/fig 8-20.png" height="200">


In [13]:
class TimeAttention:
  def __init__(self):
    self.params, self.grads = [], []
    self.layers = None
    self.attention_weight = None
    
  def forward(self, hs_enc, hs_dec): # 맥락 벡터 구함
    N, T, H = hs_dec.shape
    out = np.empty_like(hs_dec)
    self.layers = []
    self.attention_weight = []
    
    for t in range(T):
      layer = Attention()
      out[:, t, :] = layer.forward(hs_enc, hs_dec[:, t, :])
      self.layers.append(layer)
      self.attention_weight.append(layer.attention_weight)
    
    return out
  
  def backward(self, dout):
    N, T, H = dout.shape
    dhs_enc = 0
    dhs_dec = np.empty_like(dout)
    
    for t in range(T):
      layer = self.layers[t]
      dhs, dh = layer.backward(dout[:, t, :])
      dhs_enc += dhs
      dhs_dec[:, t, :] = dh
    
    return dhs_enc, dhs_dec

# 8.2 어텐션을 갖춘 seq2seq 구현

## 8.2.1 Encoder 구현

In [14]:
import sys
sys.path.append('../../modules/Part2/')
from common.time_layers import *
from seq2seq import Encoder, Seq2seq
from attention_layer import TimeAttention

class AttentionEncoder(Encoder):
  def forward(self, xs):
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    return hs
  
  def backward(self, dhs):
    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout

## 8.2.2 Decoder 구현

<img src="../../../data/deep_learning_2_images/fig 8-21.png" height="400">

In [ ]:
class AttentionDecoder:
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn
    

## 8.2.3 seq2seq 구현

In [ ]:
import sys
sys.path.append('../../modules/Part2/')

from seq2seq import Encoder, Seq2seq

class AttentionSeq2seq(Seq2seq):
  def __init__(self, vocab_size, wordvec_size, hidden_size):
    args = vocab_size, wordvec_size, hidden_size
    self.encoder = AttentionEncoder(*args)
    self.decoder = AttentionDecoder(*args)
    self.softmax = TimeSoftmaxWithLoss()
    
    self.params = 

# 8.3 어텐션 평가

- 번역용 데이터셋 중에는 'WMT'가 유명. 
  - 영어와 프랑스어 학습 데이터가 쌍으로 준비되어 있다.
  - 많은 연구에서 벤치마크로 이용됨
  - 덩치가 커서 부담됨 ( 20GB )

## 8.3.1 날짜 형식 변환 문제

- 영어권에서 사용되는 다양한 날짜 형식을 표준 형식으로 변환하는 것이 목표

<img src="../../../data/deep_learning_2_images/fig 8-22.png" height="200">

- 겉보기만큼 간단하지 않다.
  - 다양한 변형, 다양한 변환 규칙. 
- 입력과 출력 사이에 알기 쉬운 대응 관계가 있다.
  - 년-월-일의 대응관계가 존재
  - 어텐션이 각각의 원소에 올바르게 주목하고 있는지를 확인 가능

<img src="../../../data/deep_learning_2_images/fig 8-23.png" height="400">

## 8.3.2 어텐션을 갖춘 seq2seq의 학습

In [4]:
import sys
sys.path.append('../../modules/Part2/')
import numpy as np
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from attention_seq2seq import AttentionSeq2seq
from seq2seq import Seq2seq
from peeky_seq2seq import PeekySeq2seq

In [23]:
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

In [24]:
for i in range(10):
  print(''.join([id_to_char[a] for a in x_train[i]]), end=' ')
  print(''.join([id_to_char[a] for a in t_train[i]]))


2/7/72                        _1972-02-07
Apr 6, 1993                   _1993-04-06
December 13, 1999             _1999-12-13
monday, january 29, 2001      _2001-01-29
Tuesday, October 19, 1976     _1976-10-19
May 1, 1972                   _1972-05-01
12/3/87                       _1987-12-03
8/2/78                        _1978-08-02
June 25, 2006                 _2006-06-25
Jul 3, 2000                   _2000-07-03


In [25]:
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

In [11]:
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5.0

model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [26]:
acc_list = []
for epoch in range(max_epoch):
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
  
  correct_num = 0
  for i in range(len(x_test)):
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose, is_reverse=True)
  acc = float(correct_num / len(x_test))
  acc_list.append(acc)
  
  print(f"검증 정확도 : {acc * 100 :.3f}")

| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.08
| 에폭 1 |  반복 21 / 351 | 시간 6[s] | 손실 3.08
| 에폭 1 |  반복 41 / 351 | 시간 12[s] | 손실 1.90
| 에폭 1 |  반복 61 / 351 | 시간 19[s] | 손실 1.72
| 에폭 1 |  반복 81 / 351 | 시간 25[s] | 손실 1.47
| 에폭 1 |  반복 101 / 351 | 시간 31[s] | 손실 1.19
| 에폭 1 |  반복 121 / 351 | 시간 38[s] | 손실 1.14
| 에폭 1 |  반복 141 / 351 | 시간 44[s] | 손실 1.09
| 에폭 1 |  반복 161 / 351 | 시간 50[s] | 손실 1.06
| 에폭 1 |  반복 181 / 351 | 시간 57[s] | 손실 1.05
| 에폭 1 |  반복 201 / 351 | 시간 63[s] | 손실 1.04
| 에폭 1 |  반복 221 / 351 | 시간 69[s] | 손실 1.03
| 에폭 1 |  반복 241 / 351 | 시간 76[s] | 손실 1.02
| 에폭 1 |  반복 261 / 351 | 시간 83[s] | 손실 1.01
| 에폭 1 |  반복 281 / 351 | 시간 89[s] | 손실 1.01
| 에폭 1 |  반복 301 / 351 | 시간 96[s] | 손실 1.00
| 에폭 1 |  반복 321 / 351 | 시간 103[s] | 손실 1.00
| 에폭 1 |  반복 341 / 351 | 시간 109[s] | 손실 1.00
Q 10/15/94                     
T 1994-10-15
X 1977-07-11
---
Q thursday, november 13, 2008  
T 2008-11-13
X 1977-07-11
---
Q Mar 25, 2003                 
T 2003-03-25
X 1977-07-11
---
Q Tuesday, November 22, 2016

KeyboardInterrupt: 

결과비교

<img src="../../../data/deep_learning_2_images/fig 8-25.png" height="300">
<img src="../../../data/deep_learning_2_images/fig 8-26.png" height="300">


## 8.3.3 어텐션 시각화

- 어텐션이 시계열 반환을 수행할 떄, 어느 원소에 주의를 기울이는지를 눈으로 살펴보려는 시도.
- 각 시각의 어텐션 가중치를 인스턴스 변수로 보관하고 있으므로, 이를 시각화하기란 아주 간단

<img src="../../../data/deep_learning_2_images/fig 8-27.png" height="300">
<img src="../../../data/deep_learning_2_images/fig 8-28.png" height="300">

- 어텐션은 인간이 이해할 수 있는 구조나 의미를 모델에 제공
- 단어와 단어의 관련성 확인 가능
- 모델의 처리 논리가 인간의 논리를 따르는지를 판단 가능

# 8.4 어텐션에 관한 남은 이야기

## 8.4.1 양방향 RNN
<img src="../../../data/deep_learning_2_images/fig 8-29.png" height="500">

- 왼쪽에 있는 정보만 담김
- 주변정보를 균형있게 담고 싶다.

<img src="../../../data/deep_learning_2_images/fig 8-30.png" height="500">

- 역방향으로 처리하는 LSTM 계층도 추가 
- 두 LSTM 계층의 은닉 상태를 연결시킨 벡터를 최종 은닉 상태로 처리 (합하거나 평균내는 방법 등등)
- 각 단어에 대응하는 은닉 상태 벡터에는 좌와 우 양쪽 방향으로부터의 정보를 집약할 수 있다.
- 균형 잡힌 정보가 인코딩되게 된다 .

## 8.4.2 Attention 계층 사용 방법

<img src="../../../data/deep_learning_2_images/fig 8-31.png" height="400">
<img src="../../../data/deep_learning_2_images/fig 8-32.png" height="400">

- 맥락벡터가 다음 시각의 LSTM 계층에 입력되도록 연결
- 어떤 영향? -> 해보지 않으면 모른다.

## 8.4.3 seq2seq 심층화와 skip 연결

- 복잡한 문제 -> 높은 표현력을 요구 -> LSTM 계층을 깊게 쌓는다.

<img src="../../../data/deep_learning_2_images/fig 8-33.png" height="400">

- Encoder와 Decoder에서는 같은 층수의 LSTM 계층을 이용하는 것이 일반적
- Attention 계층의 사용법은 여러 변형이 존재
  - Decoder의 LSTM 계층의 은닉 상태를 Attention 계층에 입력하고,    
Attention 계층의 출력인 맥락벡터를 Decoder의 여러 계층(LSTM 계층과 Affine 계층)으로 전파
- 층을 깊게 할 떄 사용되는 중요 기법 중 skip connection (residual connection, short-cut)

<img src="../../../data/deep_learning_2_images/fig 8-34.png" height="400">

- 계층을 건너뛰는 연결. 
- skip 연결의 기울기가 아무런 영향을 받지 않고 모든 계층으로 흐름
- 층이 깊어져도 기울기가 소실되지 않고 전파되어, 결과적으로 좋은 학습을 기대할 수 있다.

# 8.5 어텐션 응용

### 8.5.1 구글 신경망 기계 번역 (GNMT)

규칙기반 번역 -> 용례 기반 번역 -> 통계 기반 번역 -> 신경망 기계 번역 (NMT, Neural Machine Translation)

<img src="../../../data/deep_learning_2_images/fig 8-35.png" height="400">

- LSTM 계층의 다층화
- 양방향 LSTM
- skip 연결
- 다수의 GPU로 분산 학습
- 낮은 빈도의 단어처리나 추론 고속화를 위한 양자화 등 다양한 연구가 이루어지고 있다.

<img src="../../../data/deep_learning_2_images/fig 8-36.png" height="400">


## 8.5.2 트랜스포머

- RNN은 이전 시각에 계산한 결과를 이용하여 순서대로 계산
- RNN의 계산을 시간 방향으로 병렬 계산하기란 불가능
- 이는 GPU를 사용한 병렬 계산 환경에서 이뤄진다는 점을 생각하면 큰 병목이다.
- RNN을 없애는 연구가 활발히 이뤄지고 있다.
- Tranformer model
- self-attention
- 하나의 시계열 데이터를 대상으로 한 어텐션으로,  
 하나의 시게열 데이터 내에서 각 원소가 다른 원소들과 어떻게 관련되는지를 살펴보자는 취지

<img src="../../../data/deep_learning_2_images/fig 8-37.png" height="200">
<img src="../../../data/deep_learning_2_images/fig 8-38.png" height="400">

- 두 입력선이 모두 하나의 시계열 데이터로부터 나온다. 
- 하나의 시계열 데이터 내에서의 원소간 대응 관계가 구해짐

<img src="../../../data/deep_learning_2_images/fig 8-39.png" height="400">

## 8.5.3 뉴럴 튜링 머신 (NTM)

- 신경망에도 외부 메모리를 이용하여 새로운 힘을 부여
- 외부 메모리를 통한 확장
- RNN, LSTM 내부 상태를 활용해 시계열 데이터를 기억.   
내부 상태는 길이가 고정이라 채워 넣을 수 있는 정보량이 제한적
- RNN 외부에 기억장치를 두고 필요한 정보를 거기에 적절하게 기록하는 방안을 착안

- RNN의 외부에 정보 저장용 메모리 기능을 배치하고,    
어텐션을 이용하여 그 메모리로부터 필요한 정보를 읽거나 쓰는 방법 

- NTM ,DeepMind , DNC라는 기법으로 개선한 논문. NTM의 메모리 조작을 더욱 강화학 버전

<img src="../../../data/deep_learning_2_images/fig 8-40.png" height="400">
<img src="../../../data/deep_learning_2_images/fig 8-41.png" height="400">
